In [55]:
#__Data Preprocessing__

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [57]:
#__Importing the dataset__
original_data = pd.read_csv('dataset.csv')

In [58]:
#__inserting random values in age__
for _i in range(len(original_data)):
    if pd.isnull(original_data.loc[_i, "age"]):
        original_data.loc[_i, "age"] = random.randint(20,85)

In [59]:
#__Spliting and removing replacing words__
column_to_add = []
for _symptom in original_data.loc[:,"symptoms"]:
   if type(_symptom) == str:
      _temp = _symptom.replace("feve","fever").replace("feaver","fever").replace("coughing","cough").replace("feverr","fever").replace("difficult in breathing","difficulty breathing")
      column_to_add.extend(_temp.split(","))
#__individual symptoms__ 
new_column = []
for _column in column_to_add:
   _temp_clm = _column.strip()
   new_column.append(_temp_clm)

#__removing duplicates__
new_column = list(dict.fromkeys(new_column))
new_column = new_column[:20]
   

In [60]:
#__adding the columns of symptoms in the main file__
for _column in new_column:
    original_data[_column] = np.nan

In [61]:
#__Adding random items from the new_column in the Symptoms__
for _i in range(len(original_data)):
    if pd.isnull(original_data.loc[_i, "symptoms"]):
        _random_int = random.randint(1,10)
        original_data.at[_i, "symptoms"] = random.sample(new_column,k=_random_int)


In [62]:
#__Addind the missing genders randomly__
_gender = ["male","female"]

for _i in range(len(original_data)):
    if pd.isnull(original_data.loc[_i, "gender"]):
        original_data.at[_i, "gender"] = random.choice(_gender)

In [63]:
#__Addind the missing death numbers randomly__
for _i in range(len(original_data)):
    if pd.isnull(original_data.loc[_i, "death"]):
        original_data.at[_i, "death"] = random.randint(0,1)

In [64]:
#__Filling all the symptoms cells__
original_data.iloc[:,5:]=0

In [65]:
#__Selecting 1 if symptoms are detected__
for _i in range(len(original_data)):
    for _each in original_data.loc[_i, "symptoms"]:
        original_data.at[_i,_each] = 1


In [66]:
#__Final processed dataset__
processed_data = original_data.drop(labels="symptoms", axis=1).iloc[:3000,:]

In [67]:
#__Independent variable__
X = processed_data.iloc[:,2:24].values

In [68]:
#__Dependent Variable (Death)__
Y = processed_data.iloc[:,1].values

In [69]:
#__Encodeing the gender__

from sklearn.preprocessing import LabelEncoder,OneHotEncoder

_labelEncoder_X_Gender = LabelEncoder()
X[:,0] = _labelEncoder_X_Gender.fit_transform(X[:,0])

_onehotencoder = OneHotEncoder(categorical_features= [0])
X = _onehotencoder.fit_transform(X).toarray()

In [70]:
#__Avoiding dummy variable trap__

X = X[:,1:]

In [71]:
#__Spliting the dataset into training set and test set__

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [72]:
#__Feature Scaling__
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_parameters = sc.fit(X_train)

X_test = sc.transform(X_test)
X_train = sc.transform(X_train)

In [73]:
#__Making the ANN__

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [74]:
#__initializing the ANN__
classifier = Sequential()

In [75]:
#__Addind the input layer and the first hidden layer__
classifier.add(Dense(output_dim=11, init='uniform', activation='relu', input_dim=22))
classifier.add(Dropout(p=.1))

#__Adding the second hidden layer__
classifier.add(Dense(output_dim=11, init='uniform', activation='relu'))
classifier.add(Dropout(p=.1))

#__Adding the second hidden layer__
classifier.add(Dense(output_dim=11, init='uniform', activation='relu'))
classifier.add(Dropout(p=.1))

#__Adding the output layer__
classifier.add(Dense(output_dim=1, init='uniform', activation='sigmoid'))

#__Compiling the ANN__
classifier.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [76]:
#__Fitting the ANN to the training set__
classifier.fit(X_train, Y_train, batch_size=25, epochs=500)

och 283/500
2400/2400 [==============================] - 0s - loss: 0.5655 - acc: 0.6871
Epoch 284/500
2400/2400 [==============================] - 0s - loss: 0.5594 - acc: 0.6854
Epoch 285/500
2400/2400 [==============================] - 0s - loss: 0.5642 - acc: 0.6733
Epoch 286/500
2400/2400 [==============================] - 0s - loss: 0.5584 - acc: 0.6783
Epoch 287/500
2400/2400 [==============================] - 0s - loss: 0.5636 - acc: 0.6821
Epoch 288/500
2400/2400 [==============================] - 0s - loss: 0.5594 - acc: 0.6717
Epoch 289/500
2400/2400 [==============================] - 0s - loss: 0.5633 - acc: 0.6867
Epoch 290/500
2400/2400 [==============================] - 0s - loss: 0.5626 - acc: 0.6775
Epoch 291/500
2400/2400 [==============================] - 0s - loss: 0.5608 - acc: 0.6838
Epoch 292/500
2400/2400 [==============================] - 0s - loss: 0.5582 - acc: 0.6750
Epoch 293/500
2400/2400 [==============================] - 0s - loss: 0.5592 - acc: 0.6804
E

In [77]:
#___Part 3 Making the predictions and evaluating the model___

#__Predicting the test set results__
y_predicted_value = classifier.predict(X_test)

In [78]:
y_predicted_boolean = (y_predicted_value>0.5)

In [79]:
#__making the error matrix also known as the confusion matrix__
from sklearn.metrics import confusion_matrix
error = confusion_matrix(Y_test,y_predicted_boolean)